In [ ]:
!pip install simpy

In [ ]:
import random
import simpy
import numpy
import matplotlib.pyplot as plt
import math

In [ ]:
class BaseDeDatos(simpy.Resource):

  def __init__(self, env, mu_demora, probabilidad = 1):
    super().__init__(env, 1)

    self.mu_demora = mu_demora
    self.probabilidad = probabilidad # Probabilidad de atender solicitud

  def get_demora_solicitud(self):
    return random.expovariate(1 / self.mu_demora)

In [ ]:
class Solicitud():

  def __init__(self, env, id):
    self.env = env
    self.id = id

  def procesar(self, bdd, estadisticas):
    tiempo_llegada = self.env.now
    estadisticas.agregar_solicitud_a_cola(tiempo_llegada)

    with bdd.request() as req:
      yield req

      tiempo_inicio_proceso = self.env.now
      estadisticas.eliminar_solicitud_de_cola(tiempo_inicio_proceso)

      tiempo_de_espera = tiempo_inicio_proceso - tiempo_llegada
      estadisticas.agregar_tiempo_de_espera(tiempo_de_espera)

      demora = bdd.get_demora_solicitud()
      yield self.env.timeout(demora)

In [ ]:
class Estadisticas():

    def __init__(self):
      self.solicitudes_en_cola = 0
      self.espera_acumulada = 0
      self.solicitudes_atendidas_sin_espera = 0
      self.tamanio_cola = [] # Tamanio de la cola en cada instante de tiempo

    def agregar_solicitud_a_cola(self, tiempo):
      self.solicitudes_en_cola += 1
      self.tamanio_cola.append((tiempo, self.solicitudes_en_cola))

    def eliminar_solicitud_de_cola(self, tiempo):
      self.solicitudes_en_cola -= 1
      self.tamanio_cola.append((tiempo, self.solicitudes_en_cola))

    def agregar_tiempo_de_espera(self, tiempo_de_espera):
      if tiempo_de_espera == 0:
        self.solicitudes_atendidas_sin_espera += 1

      self.espera_acumulada += tiempo_de_espera

    def tiempo_medio_de_espera(self):
      return self.espera_acumulada / cantidad_solicitudes

    def proporcion_solicitudes_sin_espera(self):
      return self.solicitudes_atendidas_sin_espera / cantidad_solicitudes

In [ ]:
def generar_solicitudes(env, cantidad_solicitudes, intervalo_llegadas, bases_de_datos, estadisticas):
    for i in range(cantidad_solicitudes):
        bdd = seleccionar_base_de_datos(bases_de_datos)

        solicitud = Solicitud(env, i)
        env.process(solicitud.procesar(bdd, estadisticas))

        t = random.expovariate(1.0 / intervalo_llegadas)
        yield env.timeout(t)

def seleccionar_base_de_datos(bases_de_datos):
  return numpy.random.choice(bases_de_datos, p = [bdd.probabilidad for bdd in bases_de_datos])

In [ ]:
def simular_una_base(mu_demora):
  env = simpy.Environment()

  bases_de_datos = [BaseDeDatos(env, mu_demora)]

  estadisticas = Estadisticas()
  env.process(generar_solicitudes(env, cantidad_solicitudes, intervalo_llegadas, bases_de_datos, estadisticas))
  env.run()

  return estadisticas

In [ ]:
def simular_dos_bases(mu_demora_1, mu_demora_2):
  env = simpy.Environment()

  bases_de_datos = [BaseDeDatos(env, mu_demora_1, 0.7), BaseDeDatos(env, mu_demora_2, 0.3)]

  estadisticas = Estadisticas()
  env.process(generar_solicitudes(env, cantidad_solicitudes, intervalo_llegadas, bases_de_datos, estadisticas))
  env.run()

  return estadisticas

In [ ]:
cantidad_solicitudes = 10_000
intervalo_llegadas = 4

In [ ]:
mu_demora = 0.8

estadisticas = simular_una_base(mu_demora)

print('Tiempo medio de espera: %f' % estadisticas.tiempo_medio_de_espera())
print('Proporicion de solicitudes atendidas sin espera: %f' % estadisticas.proporcion_solicitudes_sin_espera())

Tiempo medio de espera: 0.193039
Proporicion de solicitudes atendidas sin espera: 0.801100


In [ ]:
mu_demora_1 = 0.7
mu_demora_2 = 1

estadisticas = simular_dos_bases(mu_demora_1, mu_demora_2)

print('Tiempo medio de espera: %f' % estadisticas.tiempo_medio_de_espera())
print('Proporicion de solicitudes atendidas sin espera: %f' % estadisticas.proporcion_solicitudes_sin_espera())

Tiempo medio de espera: 0.088920
Proporicion de solicitudes atendidas sin espera: 0.890900
